### Imports

In [1]:
import torch
import classiq
from tqdm.auto import tqdm

import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from classiq import create_model, synthesize, show, QFunc, QArray, QBit, Output, allocate, RX, RY, RZ, RZZ, RXX, RYY, CZ
from classiq.applications.qnn import QLayer
from classiq.execution import execute_qnn
from classiq.synthesis import SerializedQuantumProgram

from classiq.applications.qnn.types import (
    MultipleArguments,
    SavedResult,
    ResultsCollection,
)

classiq.authenticate()

/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/classiq/_internals/authentication/token_manager.py:82: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


'cpu'

### Preparing Data

#### Getting a Dataset

In [3]:
def input_transform(image):
    """
    The input MNIST images are all 28 × 28 px. This function will firstly center-crop 
    them to 24 × 24 and then down-sample them to 4 × 4 for MNIST. Then we convert 
    the image pixels into angles for passing them into Rotation gates later for encoding.
    """
    image = transforms.ToTensor()(image)
    image = transforms.CenterCrop(24)(image)
    image = transforms.Resize(size = (4,4))(image)
    image = image.squeeze()
    image_pixels = torch.flatten(image)
    angles = torch.sqrt(image_pixels / 256)
    
    return angles

In [4]:
def target_transform(x):
    return x

In [5]:
# Setup training data
train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=input_transform,
    target_transform=target_transform
)

# Setup testing data
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=input_transform,
    target_transform=target_transform
)

#### Visualize the Data

In [6]:
len(train_data), len(test_data)

(60000, 10000)

In [7]:
# See the first training example
image, label = train_data[0]
image, label

/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


(tensor([0.0000, 0.0000, 0.0317, 0.0378, 0.0000, 0.0336, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0477, 0.0000, 0.0295, 0.0620, 0.0000, 0.0000]),
 5)

#### Prepare Dataloader

In [8]:
from torch.utils.data import Subset

# Define the size of the subset
subset_size = 64

# Create subsets of the datasets
train_subset = Subset(train_data, range(subset_size))
test_subset = Subset(test_data, range(subset_size))

In [9]:
# Setup the batch size hyperparameter
BATCH_SIZE = 32

# Turn datasets into iterables (batches)
train_dataloader = DataLoader(train_subset,
    batch_size=BATCH_SIZE, 
    shuffle=True
)

test_dataloader = DataLoader(test_subset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [10]:
# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7f905f89b9d0>, <torch.utils.data.dataloader.DataLoader object at 0x7f906042cfd0>)
Length of train dataloader: 2 batches of 32
Length of test dataloader: 2 batches of 32


### Quantum Model

In [11]:
@QFunc
def encoding(q: QArray[QBit]) -> None:
    """
    This function encodes the input data into the qubits. This input data is a 4x4 image pixel values 
    converted into angle for rotation gates (RX, RY, RZ, RX) in form of a 16x1 vector. 
    We encode 4 pixels per qubit.

    Args:
        q (QArray[QBit]): Array of four Qubits to encode the input data into.
    """
    RX(theta="input_0", target=q[0]) # Pixel 0 on Qubit 0
    RY(theta="input_1", target=q[0]) # Pixel 1 on Qubit 0
    RZ(theta="input_2", target=q[0]) # Pixel 2 on Qubit 0
    RX(theta="input_3", target=q[0]) # Pixel 3 on Qubit 0
    
    RX(theta="input_4", target=q[1]) # Pixel 4 on Qubit 1
    RY(theta="input_5", target=q[1]) # Pixel 5 on Qubit 1
    RZ(theta="input_6", target=q[1]) # Pixel 6 on Qubit 1
    RX(theta="input_7", target=q[1]) # Pixel 7 on Qubit 1
    
    RX(theta="input_8", target=q[2]) # Pixel 8 on Qubit 2
    RY(theta="input_9", target=q[2]) # Pixel 9 on Qubit 2
    RZ(theta="input_10", target=q[2]) # Pixel 10 on Qubit 2
    RX(theta="input_11", target=q[2]) # Pixel 11 on Qubit 2
    
    RX(theta="input_12", target=q[3]) # Pixel 12 on Qubit 3
    RY(theta="input_13", target=q[3]) # Pixel 13 on Qubit 3
    RZ(theta="input_14", target=q[3]) # Pixel 14 on Qubit 3
    RX(theta="input_15", target=q[3]) # Pixel 15 on Qubit 3


In [12]:
@QFunc
def mixing(q: QArray[QBit]) -> None:
    """
    This function performs the mixing operation on the qubits. 
    This is done by applying a series of RZZ, RXX, RYY gates to form a
    ring connection.

    Args:
        q (QArray[QBit]): Array of four Qubits to apply the mixing operation on.
    """
    RZZ(theta="weight_0", target=q[0:2])
    RZZ(theta="weight_1", target=q[1:3])
    RZZ(theta="weight_2", target=q[2:4])
    
    RXX(theta="weight_4", target=q[0:2])
    RXX(theta="weight_5", target=q[1:3])
    RXX(theta="weight_6", target=q[2:4])
    
    RYY(theta="weight_8", target=q[0:2])
    RYY(theta="weight_9", target=q[1:3])
    RYY(theta="weight_10", target=q[2:4])

In [13]:
@QFunc
def cz_block(q: QArray[QBit]) -> None:
    """
    This function applies CZ gates between each qubit.

    Args:
        q (QArray[QBit]): Array of four Qubits to apply the entanglement operation on.
    """
    CZ(control=q[0], target=q[1])
    CZ(control=q[1], target=q[2])
    CZ(control=q[2], target=q[3])

In [14]:
@QFunc
def main(res: Output[QArray[QBit]]) -> None:
    """
    This is the main function from which model will be created. 
    It calls the other functions to perform the encoding, mixing and entanglement.

    Args:
        res (Output[QArray[QBit]]): Output QArray of QBits from which the model will be created.
    """
    allocate(4, res)
    encoding(q=res)
    mixing(q=res)
    cz_block(q=res)

In [15]:
# Create a model
model = create_model(main)
quantum_program = synthesize(model)
show(quantum_program)

Opening: https://platform.classiq.io/circuit/79ef146a-a004-4a95-ba01-72651b5bc3a5?version=0.33.0


### Quantum Neural Network

In [16]:
def execute(quantum_program: SerializedQuantumProgram, arguments: MultipleArguments) -> ResultsCollection:
    return execute_qnn(quantum_program, arguments)

In [17]:
def post_process(result: SavedResult) -> torch.Tensor:
    counts: dict = result.value.counts
    # The probability of measuring |0>
    # print(f"counts: {counts}")
    p_zero: float = counts.get("0", 0.0) / sum(counts.values())
    return torch.tensor(p_zero)

In [18]:
class Net(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__()
        self.qlayer = QLayer(
            quantum_program,
            execute,
            post_process,
            *args,
            **kwargs
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.qlayer(x)
        return x

In [19]:
qnn = Net()

gio: https://platform.classiq.io/circuit/79ef146a-a004-4a95-ba01-72651b5bc3a5?version=0.33.0: Operation not supported


### Training and Testing Loop

In [20]:
_LEARNING_RATE = 1.0

# choosing our loss function
loss_fn = nn.L1Loss()
# choosing our optimizer
optimizer = optim.SGD(qnn.parameters(), lr=_LEARNING_RATE)

In [21]:
def train(
    model: nn.Module,
    data_loader: DataLoader,
    loss_fn: nn.modules.loss._Loss,
    optimizer: optim.Optimizer,
    epochs: int = 20,
) -> None:
    train_loss = 0
    model.to(device)
    for epoch in tqdm(range(epochs)):
        print(f"Epoch: {epoch}\n----------")
        for batch, (data, label) in enumerate(data_loader):
            # Send data to device (GPU or CPU)
            data, label = data.to(device), label.to(device)
            
            # 1. Forward pass
            output = model(data)
            
            # 2. Calculate loss
            loss = loss_fn(output, label)
            train_loss += loss
            
            # 3. Optimizer zero grad
            optimizer.zero_grad()
            
            # 4. Loss backward
            loss.backward()
            
            # 5. Optimizer step
            optimizer.step()
    
        # Calculate loss per epoch and print out what's happening
        train_loss /= len(data_loader)
        print(f"Train loss: {train_loss:.5f}")

In [22]:
train(qnn, train_dataloader, loss_fn, optimizer, epochs=20)

  0%|          | 0/20 [00:00<?, ?it/s]/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch: 0
----------


  5%|▌         | 1/20 [00:59<18:48, 59.38s/it]

Train loss: 4.43750
Epoch: 1
----------


 10%|█         | 2/20 [02:02<18:28, 61.57s/it]

...

Train loss: 8.87497
Epoch: 18
----------


 95%|█████████▌| 19/20 [18:23<00:59, 59.30s/it]

Train loss: 8.87498
Epoch: 19
----------


100%|██████████| 20/20 [19:28<00:00, 58.42s/it]

Train loss: 8.87499


In [23]:
def test(
    model: nn.Module,
    data_loader: DataLoader,
    atol=1e-4
) -> float:
    num_correct = 0
    total = 0
    
    # Put the model in eval mode
    model.eval()
    
    # Turn on inference mode context manager
    with torch.inference_mode():
        for data, labels in data_loader:
            # Send data to GPU
            data, labels = data.to(device), labels.to(device)
            
            # 1. Forward pass: Let the model predict
            predictions = model(data)
            
            # Get a tensor of booleans, indicating if each label is close to the real label
            is_prediction_correct = torch.isclose(predictions, labels.type(torch.float32), atol=atol)
            print("Label: ", labels)
            print("predictions: ", predictions)
            print("is_prediction_correct: ", is_prediction_correct)
            
            # Count the amount of `True` predictions
            num_correct += is_prediction_correct.sum().item()
            
            # Count the total evaluations
            #   the first dimension of `labels` is `batch_size`
            total += labels.size(0)
    
    # Calculate the accuracy
    accuracy = float(num_correct) / float(total)
    print(f"Test Accuracy of the model: {accuracy*100:.2f}")
    return accuracy

In [24]:
test(qnn, test_dataloader)

Label:  tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1])
predictions:  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
is_prediction_correct:  tensor([False, False, False,  True, False, False, False, False, False, False,
         True, False, False,  True, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False,  True, False,
        False, False])
Label:  tensor([3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0,
        4, 1, 9, 5, 7, 8, 9, 3])
predictions:  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
is_prediction_correct:  tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, 

0.09375